In [1]:
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras import datasets as tfd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd

In [2]:
def scheduler(epoch, lr):
   if epoch < 10:
     return lr
   else:
     return lr * tf.math.exp(-0.1)

In [3]:
train_data, test_data = tfd.cifar100.load_data(label_mode="coarse")
(x_train, y_train), (x_test, y_test) = train_data, test_data
x_train, x_test = np.mean(x_train, axis=3), np.mean(x_test, axis=3)

In [4]:
N_CLASSES = 20
INPUT_SIZE = (32, 32, 1)
LOSS = 'categorical_crossentropy'
METRICS = ['accuracy']
CALLBACKS = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3), tf.keras.callbacks.LearningRateScheduler(scheduler)] # Correct?
BATCH_SIZE = 32
EPOCHS = 10

In [5]:
#Set shape to (num_samples, h, w, numchannels) = (40000, 32, 32, 1)
x_train = np.expand_dims(x_train, axis=-1) 
x_test = np.expand_dims(x_test, axis=-1)

# Convert labels to one-of-K (one-hot) encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=N_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=N_CLASSES)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

TRAIN_SIZE, _ , _ , _ = x_train.shape
input_img = tf.keras.Input(shape=INPUT_SIZE)
y_pred = None

In [7]:
def task1():
    fig, axs = plt.subplots(4, 5)
    axs = axs.flatten()
    for i, ax in enumerate(axs):
        ax.imshow(x_train[i], cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()
# task1()

In [6]:
import tensorflow as tf

def M5(l1_value, l2_value):
    global y_pred
    # Double number of filters on each iteration - generic first layers, specific later layers 
    reg = tf.keras.regularizers.L1L2(l1=l1_value, l2=l2_value)
    h0 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), kernel_regularizer=reg)(input_img)
    h0 = tf.keras.layers.MaxPool2D((2, 2))(h0)

    h1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding='same', kernel_regularizer=reg)(h0)
    h1 = tf.keras.layers.MaxPool2D((2, 2))(h1)

    h2 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same', kernel_regularizer=reg)(h1)
    h2 = tf.keras.layers.MaxPool2D((2, 2))(h2)

    fh1 = tf.keras.layers.Flatten()(h2)
    fh2 = tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer=reg)(fh1)
    fh3 = tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=reg)(fh2)
    y_pred = tf.keras.layers.Dense(units=N_CLASSES, activation='softmax')(fh3)



In [30]:
M5()
model_name = 'M5-regularized'

model = tf.keras.Model(input_img, y_pred)
# model.summary()

x_train_n = x_train
x_val_n = x_val

x_train_n = (x_train.astype('float32')) / 255
x_val_n = (x_val.astype('float32')) / 255

sampler = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                                          width_shift_range=0.1, height_shift_range=0.1).flow(x_train_n, y_train, batch_size=BATCH_SIZE)


optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer, loss=LOSS, metrics=METRICS)
history = model.fit(sampler, epochs=EPOCHS, steps_per_epoch=TRAIN_SIZE//BATCH_SIZE, callbacks=CALLBACKS, validation_data=(x_val_n, y_val))
history_df = pd.DataFrame(history.history)
history_df.to_csv(f'{model_name}_training_history.csv', index=False)
test_loss, test_acc = model.evaluate(x_val_n, y_val)
evaluation_df = pd.DataFrame({'Test Loss': [test_loss], 'Test Accuracy': [test_acc]})
evaluation_df.to_csv(f'{model_name}_evaluation_results.csv', index=False)


Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 1)]       0         
                                                                 
 conv2d_26 (Conv2D)          (None, 30, 30, 32)        320       
                                                                 
 max_pooling2d_24 (MaxPooli  (None, 15, 15, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_27 (Conv2D)          (None, 15, 15, 128)       36992     
                                                                 
 max_pooling2d_25 (MaxPooli  (None, 7, 7, 128)         0         
 ng2D)                                                           
                                                                 
 conv2d_28 (Conv2D)          (None, 7, 7, 256)         2951

KeyboardInterrupt: 

In [11]:

x_train_n = (x_train.astype('float32')) / 255
x_val_n = (x_val.astype('float32')) / 255


for l1 in [10**(-i) for i in range(1, 11)]:
    for l2 in [10**(-i) for i in range(1, 11)]:
        M5(l1, l2)
        model_name = f'M5-l1_{l1}_l2_{l2}'

        
        model = tf.keras.Model(input_img, y_pred)
        sampler = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                                            width_shift_range=0.1, height_shift_range=0.1).flow(x_train_n, y_train, batch_size=BATCH_SIZE)


        optimizer = tf.keras.optimizers.Adam()

        model.compile(optimizer=optimizer, loss=LOSS, metrics=METRICS)
        history = model.fit(sampler, epochs=EPOCHS, steps_per_epoch=TRAIN_SIZE//BATCH_SIZE, callbacks=CALLBACKS, validation_data=(x_val_n, y_val))
        history_df = pd.DataFrame(history.history)
        history_df.to_csv(f'{model_name}_training_history.csv', index=False)
        test_loss, test_acc = model.evaluate(x_val_n, y_val)
        evaluation_df = pd.DataFrame({'Test Loss': [test_loss], 'Test Accuracy': [test_acc]})
        evaluation_df.to_csv(f'{model_name}_evaluation_results.csv', index=False)

        plt.plot(history.history['val_loss'])
        plt.title('Validation loss history')
        plt.ylabel('Loss value')
        plt.xlabel('No. epoch')
        plt.savefig(f'{model_name}_val_loss.png')  # Save with the model name
        plt.close()  # Close the plot to avoid overlapping plots

        # Plot history: Accuracy
        plt.plot(history.history['val_accuracy'])
        plt.title('Validation accuracy history')
        plt.ylabel('Accuracy value (%)')
        plt.xlabel('No. epoch')
        plt.savefig(f'{model_name}_val_accuracy.png')  # Save with the model name
        plt.close()  # Close the plot

Epoch 1/10
1250/1250 [==============================] - 64s 51ms/step - loss: 53.2222 - accuracy: 0.0481 - val_loss: 14.4787 - val_accuracy: 0.0500 - lr: 0.0010
Epoch 2/10
1250/1250 [==============================] - 67s 54ms/step - loss: 14.6505 - accuracy: 0.0481 - val_loss: 14.8288 - val_accuracy: 0.0500 - lr: 0.0010
Epoch 3/10
1250/1250 [==============================] - 66s 52ms/step - loss: 14.9009 - accuracy: 0.0483 - val_loss: 14.9804 - val_accuracy: 0.0500 - lr: 0.0010
Epoch 4/10
313/313 [==============================] - 4s 14ms/step - loss: 14.7778 - accuracy: 0.0500
Epoch 1/10
1250/1250 [==============================] - 66s 52ms/step - loss: 52.2854 - accuracy: 0.0474 - val_loss: 14.7202 - val_accuracy: 0.0500 - lr: 0.0010
Epoch 2/10
1250/1250 [==============================] - 67s 54ms/step - loss: 14.6424 - accuracy: 0.0487 - val_loss: 14.4946 - val_accuracy: 0.0500 - lr: 0.0010
Epoch 3/10
1250/1250 [==============================] - 70s 56ms/step - loss: 14.5138 - accur

In [8]:
x_train_n = x_train
x_val_n = x_val



for l1 in [10**(-i) for i in range(0, 11)]:
    for l2 in [10**(-i) for i in range(0, 11)]:
        M5(l1, l2)
        model_name = f'M5-no-norm-l1_{l1}_l2_{l2}'

        
        model = tf.keras.Model(input_img, y_pred)
        sampler = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                                            width_shift_range=0.1, height_shift_range=0.1).flow(x_train_n, y_train, batch_size=BATCH_SIZE)


        optimizer = tf.keras.optimizers.Adam()

        model.compile(optimizer=optimizer, loss=LOSS, metrics=METRICS)
        history = model.fit(sampler, epochs=EPOCHS, steps_per_epoch=TRAIN_SIZE//BATCH_SIZE, callbacks=CALLBACKS, validation_data=(x_val_n, y_val))
        history_df = pd.DataFrame(history.history)
        history_df.to_csv(f'{model_name}_training_history.csv', index=False)
        test_loss, test_acc = model.evaluate(x_val_n, y_val)
        evaluation_df = pd.DataFrame({'Test Loss': [test_loss], 'Test Accuracy': [test_acc]})
        evaluation_df.to_csv(f'{model_name}_evaluation_results.csv', index=False)

        plt.plot(history.history['val_loss'])
        plt.title('Validation loss history')
        plt.ylabel('Loss value')
        plt.xlabel('No. epoch')
        plt.savefig(f'{model_name}_val_loss.png')  # Save with the model name
        plt.close()  # Close the plot to avoid overlapping plots

        # Plot history: Accuracy
        plt.plot(history.history['val_accuracy'])
        plt.title('Validation accuracy history')
        plt.ylabel('Accuracy value (%)')
        plt.xlabel('No. epoch')
        plt.savefig(f'{model_name}_val_accuracy.png')  # Save with the model name
        plt.close()  # Close the plot

Epoch 1/10
1250/1250 [==============================] - 84s 67ms/step - loss: 541.8098 - accuracy: 0.0485 - val_loss: 116.5973 - val_accuracy: 0.0500 - lr: 0.0010
Epoch 2/10
1250/1250 [==============================] - 79s 63ms/step - loss: 117.8778 - accuracy: 0.0495 - val_loss: 119.6285 - val_accuracy: 0.0500 - lr: 0.0010
Epoch 3/10
1250/1250 [==============================] - 91s 73ms/step - loss: 120.3782 - accuracy: 0.0470 - val_loss: 120.8063 - val_accuracy: 0.0500 - lr: 0.0010
Epoch 4/10
313/313 [==============================] - 7s 21ms/step - loss: 120.0048 - accuracy: 0.0500
Epoch 1/10
1250/1250 [==============================] - 99s 78ms/step - loss: 519.2474 - accuracy: 0.0477 - val_loss: 118.9494 - val_accuracy: 0.0500 - lr: 0.0010
Epoch 2/10
1250/1250 [==============================] - 88s 70ms/step - loss: 118.1752 - accuracy: 0.0487 - val_loss: 116.6285 - val_accuracy: 0.0500 - lr: 0.0010
Epoch 3/10
1250/1250 [==============================] - 88s 70ms/step - loss: 116.